In [ ]:
# If gspx is not installed, we add it to the path
import os, sys
gdir = os.path.dirname(os.getcwd())  # parent folder
sys.path.insert(0, gdir)

In [ ]:
import numpy as np
from pyquaternion import Quaternion
from gspx.signals import QuaternionSignal
from gspx.datasets import uk_weather

from gspx.utils.display import plot_graph
from gspx.datasets import WeatherGraphData, SocialGraphData
from gspx.qgsp import create_quaternion_weights, QGFT, QMatrix
from gspx.adaptive import QLMS

In [ ]:
# uk_data = WeatherGraphData()
# Ar, _ = uk_data.graph
# s = uk_data.signal

# df = uk_weather()

data = SocialGraphData()
df = data.data
Ar, _ = data.graph
s = data.signal

In [ ]:
Aq = create_quaternion_weights(
    Ar, df, icols=['median_household_income_2017'],
    jcols=['unemployment_rate_2017'],
    kcols=['uninsured_2017'], hermitian=False)

qgft = QGFT()
qgft.fit(Aq)

In [ ]:
obj = Ar
np.all(np.abs(obj - obj.transpose()) < 1e-8)

In [ ]:
qgft.fit(Aq)

In [ ]:
eigq, Vq = Aq.eigendecompose()

In [ ]:
Vq_inv = Vq.inv(assert_cond=False)

In [ ]:
QMatrix.unstack_complex_eig_matrix(V, eig)

In [ ]:
obj = Aq.complex_adjoint
np.all((obj - np.abs(obj.conjugate().transpose())) < 1e-8)

In [ ]:
x = np.linalg.inv(Aq.complex_adjoint)

In [ ]:
x

In [ ]:
How do I get the inverse of Vq out of the inverse of V?

How do I get the inverse of Vq out of the inverse of V?

Vq: eigenvectors from the quaternion adj matrix
V: eigenvectors from its complex adjoint.

In [ ]:
ca = Aq.complex_adjoint

In [ ]:
eig, V = np.linalg.eigh(ca)

In [ ]:
np.linalg.cond(ca)

In [ ]:
import sys
1 / sys.float_info.epsilon

In [ ]:
import matplotlib.pyplot as plt

Vinv = np.linalg.inv(V)

In [ ]:

# Heat kernel in all 4 quaternion dimensions
k = 0.2
ss = np.zeros(len(qgft.idx_freq))
ss[qgft.idx_freq] = np.exp(-k * np.arange(len(qgft.idx_freq)))

ss = QuaternionSignal.from_rectangular(
    np.hstack([ss[:, np.newaxis]] * 4)
)

rnd = np.random.default_rng(seed=42)
err_amplitude = 0.15

nn = QuaternionSignal.from_equal_dimensions(
    rnd.uniform(low=-err_amplitude, high=err_amplitude, size=len(ss))
)

s = qgft.inverse_transform(ss)

# Ideal LPF
h_ideal = np.zeros(len(qgft.idx_freq))
bandwidth = int(len(qgft.idx_freq) / 5)
h_ideal[qgft.idx_freq[:bandwidth]] = 1
h_idealq = QuaternionSignal.from_rectangular(np.hstack((
    h_ideal[:, np.newaxis],
    np.zeros(len(qgft.idx_freq))[:, np.newaxis],
    np.zeros(len(qgft.idx_freq))[:, np.newaxis],
    np.zeros(len(qgft.idx_freq))[:, np.newaxis]
)))

X = QMatrix.vander(qgft.eigq, 7, increasing=True)
y = h_idealq

qlms = QLMS(alpha=[0.3])
qlms.fit(X, y)
assert 'result' in qlms.res_[qlms.best_lr_]

h_opt = qlms.predict(X)
h_opt = QuaternionSignal.from_samples(h_opt.matrix.ravel())

# Ideal filter
sn = qgft.inverse_transform(ss + nn)
mse_prior = np.mean((s - sn).abs()**2)
np.testing.assert_allclose(
    mse_prior, 0.029666218144683097, rtol=1e-06, atol=1e-06)

ssn_lpf = (ss + nn).hadamard(h_idealq)
s_lpf = qgft.inverse_transform(ssn_lpf)
mse_post = np.mean((s - s_lpf).abs()**2)
np.testing.assert_allclose(
    mse_post, 0.006566188086831742, rtol=1e-06, atol=1e-06)

# FIR filter
sn = qgft.inverse_transform(ss + nn)

ssn_lpf = (ss + nn).hadamard(h_opt)
s_lpf = qgft.inverse_transform(ssn_lpf)
mse_post = np.mean((s - s_lpf).abs()**2)